In [1]:
# pandas analysis
# 数据聚合和分组操作

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
# 对数据集进行分组并对各组应用一个函数（无论是聚合还是转换） ， 通常是数据分析工作中的重要环节
#   在将数据集加载、 融合、 准备好之后， 通常就是计算分组统计或生成透视表。 
# pandas 提供# 了一个灵活高效的 gruopby 功能，能以一种自然的方式对数据集进行切片 切块 摘要等操作。

# 关系型数据库 和 SQL（Structured Query Language， 结构化查询语言） 
#    能够如此流行的原因之一就是其能够方便地对数据进行连接、过滤、转换和聚合
# 但是， 像 SQL 这样的查询语言所能执行的分组运算的种类很有限
# 由于 Python 和 pandas 强大的表达能力，可以执行复杂得多的分组运算（利用任何可以接受pandas对象或NumPy数组的函数） 。 在本章中， 你
#     1、使用一个或多个键（形式可以是函数、 数组或DataFrame列名） 分割pandas对象
#     2、计算分组的概述统计， 比如数量、 平均值或标准差， 或是用户定义的函数
#     3、应用组内转换或其他运算， 如规格化、 线性回归、 排名或选取子集等
#     4、计算透视表或交叉表
#     5、执行分位数分析以及其它统计分组分析
#     6、对时间序列数据的聚合（groupby的特殊用法之一） 也称作重采样（resampling）

In [5]:
# Groupby 机制

# Hadley Wickham（许多热门R语言包的作者） 创造了一个用于表示分组运算的术语
#       "split-applycombine"（拆分－应用－合并） 
#   第一个阶段， pandas 对象（无论是 Series、 DataFrame 还是其他的）中的数据
#              会根据你所提供的一个或多个键被拆分（split） 为多组
#             拆分操作是在对象的特定轴上执行的
#             DataFrame 可以在其行（axis=0） 或列（axis=1） 上进行分组
#   然后，将一个函数应用（apply） 到各个分组并产生一个新值
#   最后， 所有这些函数的执行结果会被合并（combine） 到最终的结果对象中
# 结果对象的形式一般取决于数据上所执行的操作

# 如下图所示

![self](./../image/groupby.png)

In [6]:
# 分组键可以有多种形式， 且类型不必相同：
#    - 列表或数组， 其长度与待分组的轴一样。
#    - 表示DataFrame某个列名的值。
#    - 字典或Series， 给出待分组轴上的值与分组名之间的对应关系。
#    - 函数， 用于处理轴索引或索引中的各个标签。

# 注意， 后三种都只是快捷方式而已， 其最终目的仍然是产生一组用于拆分对象的值。 

# 一个简单的表格型数据集（以DataFrame的形式）
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.085855,-2.622739
1,a,two,0.101006,2.430120
2,b,one,0.850283,0.551016
3,b,two,0.089985,-0.892486
4,a,one,1.277483,0.304751


In [7]:
# 假设想要按照 key1 进行分组，并计算 data1 的平均值
# 实现方式：访问 data1 ，并根据 key1 调用 groupby
grouped = df['data1'].groupby(df['key1'])
grouped

In [8]:
type(grouped)

pandas.core.groupby.generic.SeriesGroupBy

In [10]:
# 变量 grouped 是一个 GroupBy 对象，实际并未进行任何计算，只是含有有关分组键 df['key1'] 的中间数据
# 简单来说，就是该对象已经有了对各组分组执行运算所需的一切信息
# 调用 GroupBy 的 mean 方法来计算分组平均值
grouped.mean()

key1
a    0.488115
b    0.470134
Name: data1, dtype: float64

In [11]:
# 详细讲解 .mean()的调用过程
# 这里最重要的是， 数据（Series） 根据分组键进行了聚合， 产生了一个新的 Series，
# 其索引为 key1 列中的唯一值, 之所以结果中索引的名称为 key1，是因为原始 DataFrame 的列 df['key1'] 就叫这个名字

# 如果一次传入多个数组的列表，也就是通过多个键进行分组，得到的 Series 是一个层次化索引
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.681669
      two     0.101006
b     one     0.850283
      two     0.089985
Name: data1, dtype: float64

In [12]:
means.unstack()

key2,one,two
key1,,
a,0.681669,0.101006
b,0.850283,0.089985


In [13]:
# 在这个例子中， 分组键均为 Series
#  实际上， 分组键可以是任何长度适当的数组：
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.101006
            2006    0.850283
Ohio        2005    0.087920
            2006    1.277483
Name: data1, dtype: float64

In [14]:
# 通常，分组信息就位于相同的要处理 DataFrame中
# 还可以将列名（可以是字符串、 数字 或其他Python对象） 用作分组键
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.488115,0.037377
b,0.470134,-0.170735


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.681669 -1.158994
     two   0.101006  2.430120
b    one   0.850283  0.551016
     two   0.089985 -0.892486

In [16]:
# 可能已经注意到了， 第一个例子在执行 df.groupby('key1').mean() 时，
#    结果中没有 key2 列, 这是因为 df['key2'] 不是数值数据（俗称“麻烦列”），所以被从结果中排除了
# 默认情况下， 所有数值列都会被聚合，虽然有时可能会被过滤为一个子集

# noting: 任何分组关键词中的缺失值，都会被从结果中除去

# 无论准备拿 groupby 做什么，都有可能会用到 GroupBy 的 size 方法，可以返回一个含有分组大小的 Series：
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [17]:
# 对分组进行迭代

# GroupBy 对象支持迭代，可以产生一组二元元组（有分组名和数据块组成）
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.085855 -2.622739
1    a  two  0.101006  2.430120
4    a  one  1.277483  0.304751
b
  key1 key2     data1     data2
2    b  one  0.850283  0.551016
3    b  two  0.089985 -0.892486


In [18]:
# 对于多重键的情况， 元组的第一个元素将会是由键值组成的元组：
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.085855 -2.622739
4    a  one  1.277483  0.304751
('a', 'two')
  key1 key2     data1    data2
1    a  two  0.101006  2.43012
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.850283  0.551016
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.089985 -0.892486


In [19]:
# 当然，可以对这些数据片段做任何操作
# 有一个可能会觉得有用的运算： 将这些数据片段做成一个字典：
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.085855 -2.622739
 1    a  two  0.101006  2.430120
 4    a  one  1.277483  0.304751,
 'b':   key1 key2     data1     data2
 2    b  one  0.850283  0.551016
 3    b  two  0.089985 -0.892486}

In [20]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.850283,0.551016
3,b,two,0.089985,-0.892486


In [21]:
# groupby 默认是在 axis=0 （行）进行分组的， 通过设置也可以在其他任何轴上进行分组
# 拿上面例子中的 df 来说，可以根据 dtype 对列进行分组：
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [23]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.085855 -2.622739
1  0.101006  2.430120
2  0.850283  0.551016
3  0.089985 -0.892486
4  1.277483  0.304751
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


In [24]:
# 由于可能不太熟悉书中使用的一些有关编程和数据科学方面的常用术语， 所以在这里先给出其简单定义：
#     1、数据规整（Munge/Munging/Wrangling） 指的是将非结构化和（或） 散乱数据处理为结构化或整
#          洁形式的整个过程。 这几个词已经悄悄成为当今数据黑客们的行话了。 Munge这个词跟Lunge押韵。
#     2、伪码（Pseudocode） 算法或过程的“代码式”描述， 而这些代码本身并不是实际有效的源代码。
#     3、语法糖（Syntactic sugar） 这是一种编程语法， 它并不会带来新的特性， 但却能使代码更易读、更易写。

In [25]:
# 选取一列或列的子集

# 对于由 DataFrame 产生的 GroupBy 对象， 
#  如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。 
#  也就是说：
# df.groupby('key1')['data1']
# df.groupby('key1')[['data2']]

# 代码的语法糖：也就是从 Python 语法角度说应该是下面代码，从代码可读性和可理解性应该是上面代码

# df['data1'].groupby(df['key1'])
# df[['data2']].groupby(df['key1'])

In [26]:
# 尤其对于大数据集，很可能只需要对部分列进行聚合
#  例如，在前面那个数据集中，如果只需计算 data2 列的平均值并以 DataFrame 形式得到结果
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.158994
     two   2.430120
b    one   0.551016
     two  -0.892486

In [28]:
# 这种索引操作所返回的对象是一个已分组的 DataFrame（如果传入的是列表或数组） 
#          或已分组的 Series（如果传入的是标量形式的单个列名） ：
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     one    -1.158994
      two     2.430120
b     one     0.551016
      two    -0.892486
Name: data2, dtype: float64

In [30]:
# 通过 字典 或 Series 进行分组
# 除数组以外， 分组信息还可以其他形式存在

people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,2.315610,-0.165255,-1.052396,0.709598,0.543279
Steve,1.814661,-0.788101,-1.117939,-1.852481,-1.620563
Wes,0.065038,NaN,NaN,-1.234193,1.250431
Jim,-1.494505,-0.687324,2.593953,0.469832,1.585922
Travis,-0.059679,1.422576,-0.143448,1.052608,-1.078617


In [31]:
# 现在， 假设已知列的分组关系，并希望根据分组计算列的和：
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

# 现在，可以将这个字典传给 groupby，来构造数组，
# 可以直接传递字典（包含了键 “f” 来强调， 存在未使用的分组键是可以的） ：
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.342798,2.693634
Steve,-2.970419,-0.594003
Wes,-1.234193,1.315469
Jim,3.063784,-0.595907
Travis,0.909160,0.284279


In [32]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [33]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [34]:
# 通过函数进行分组

# 比起使用字典或 Series，使用 Python 函数是一种更原生的方法定义分组映射
#  任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称

#  具体点说，以上一小节的示例 DataFrame 为例，其索引值为人的名字
# 可以计算一个字符串长度的数组，更简单的方法是传入 len 函数
people.groupby(len).sum()

,a,b,c,d,e
3,0.886143,-0.852580,1.541557,-0.054764,3.379632
5,1.814661,-0.788101,-1.117939,-1.852481,-1.620563
6,-0.059679,1.422576,-0.143448,1.052608,-1.078617


In [35]:
# 将函数跟数组、列表、字典、Series 混合使用也不是问题，因为任何东西在内部都会被转换为数组
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.065038 -0.165255 -1.052396 -1.234193  0.543279
  two -1.494505 -0.687324  2.593953  0.469832  1.585922
5 one  1.814661 -0.788101 -1.117939 -1.852481 -1.620563
6 two -0.059679  1.422576 -0.143448  1.052608 -1.078617

In [36]:
# 根据索引级别分组

# 层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                     names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.245885 -0.329153  0.092811  1.037349  1.482598
1     -1.812302 -0.270322  0.319838  1.524951 -0.912206
2      1.447077  0.767615  2.547866  0.639822 -0.085907
3     -0.521469  2.705246 -1.185670 -1.297573 -0.212007

In [37]:
# 要根据级别分组， 使用level关键字传递级别序号或名字：
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [39]:
# 数据聚合

# 聚合指的是任何能够从数组产生标量值的数据转换过程
#  比如 mean、count、min 以及 sum 等

# 可能想知道在 GroupBy 对象上调用 mean() 时究竟发生了什么
#  许多常见的聚合运算都有进行优化。 然而， 除了这些方法，还可以使用其它的

# 函数名        说明
# count         分组中非NA值的数量
# sum           非 NA 值的和
# mean          非 NA 值的平均值
# median        非 NA 值的算术中位数
# std. var      无偏(分母为n-1)标准差和方差
# min、max      非 NA 值的最小值和最大值
# prod          非 NA 值的积
# first、last   第一个和最后一个非NA值

# 可以使用自己发明的聚合运算， 还可以调用分组对象上已经定义好的任何方法
#    例如， quantile 可以计算 Series 或 DataFrame 列 的样本分位数。
# 虽然 quantile 并没有明确地实现于 GroupBy， 但它是一个 Series 方法， 所以这里是能用的
# 实际上， GroupBy 会高效地对 Series 进行切片，然后对各片调用 piece.quantile(0.9)，最后将这些结果组装成最终结果
df

,key1,key2,data1,data2
0,a,one,0.085855,-2.622739
1,a,two,0.101006,2.430120
2,b,one,0.850283,0.551016
3,b,two,0.089985,-0.892486
4,a,one,1.277483,0.304751


In [40]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.042188
b    0.774253
Name: data1, dtype: float64

In [43]:
# 笔记： 自定义聚合函数要比那些经过优化的函数慢得多
#       这是因为在构造中间分组数据块时存在非常大的开销（函数调用、 数据重排等）

# 如果要使用自己的聚合函数， 只需将其传入 aggregate 或 agg 方法即可：
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.191629,5.052859
b,0.760298,1.443502


In [44]:
# 注意， 有些方法（如describe）也是可以用在这里的，即使严格来讲，它们并非聚合运算：
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.488115  0.683655  0.085855  0.093431  0.101006  0.689245   
b      2.0  0.470134  0.537612  0.089985  0.280060  0.470134  0.660209   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.277483   3.0  0.037377  2.537018 -2.622739 -1.158994  0.304751   
b     0.850283   2.0 -0.170735  1.020710 -0.892486 -0.531611 -0.170735   

                          
           75%       max  
key1                      
a     1.367435  2.430120  
b     0.190140  0.551016

In [45]:
# 面对列的多函数应用

# 回到 小费 例子，
# 导入 数据 后，添加一个小费百分比的列 tip_pct
dataset_path = './../dataset/'
tips = pd.read_csv(dataset_path + 'tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [46]:
# 已经看到， 对 Series 或 DataFrame 列的聚合运算其实就是使用 aggregate（使用自定义函数） 
# 或调用诸如 mean、 std 之类的方法。
# 可能希望对不同的列使用不同的聚合函数， 或一次应用多个函数,其实这也好办， 通过一些示例来进行讲解
# 首先，根据 天 和 smoker 对 tips 进行分组：
grouped = tips.groupby(['day', 'smoker'])

grouped_pct = grouped['tip_pct']

grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [48]:
# 如果传入一组函数 或 函数名，得到的 DataFrame 的列就会以相应的函数命名：
# 这里， 我们传递了一组聚合函数进行聚合， 独立对数据分组进行评估
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [50]:
# 并非一定要接受 GroupBy 自动给出的那些列名， 特别是 lambda 函数， 它们的名称是 ''， 
#    这样的辨识度就很低了（通过函数的 name 属性看看就知道了）
# 因此， 如果传入的是一个由 (name,function) 元组组成的列表，
#    则各元组的第一个元素就会被用作 DataFrame 的列名（可以将这种二元元组列表看做一个有序映射） 
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [53]:
# 对于 DataFrame， 还有更多选择， 
# 可以定义一组应用于全部列的一组函数， 或不同的列应用不同的函数

# 假设想要对 tip_pct 和 total_bill 列计算三个统计信息
functions = ['count', 'mean', 'max']

result = grouped['tip_pct', 'total_bill'].agg(functions)

result

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [54]:
# 如你所见， 结果 DataFrame 拥有层次化的列， 
#           这相当于分别对各列进行聚合， 然后用 concat 将结果组装到一起， 使用列名用作 keys 参数
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [55]:
# 跟前面一样， 这里也可以传入带有自定义名称的一组元组：
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]

grouped['tip_pct', 'total_bill'].agg(ftuples)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [56]:
# 现在， 假设想要对一个列或不同的列应用不同的函数
#  具体的办法是向 agg 传入一个从列名映射到函数的字典：
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [57]:
# 只有将多个函数应用到至少一列时， DataFrame才会拥有层次化的列
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [58]:
# 到目前为止， 所有示例中的聚合数据都有由唯一的分组键组成的索引（可能还是层次化的）
#  由于并不总是需要如此， 所以可以向 groupby 传入 as_index=False 以禁用该功能：
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [59]:
# 当然， 对结果调用 reset_index 也能得到这种形式的结果
# 使用 as_index=False 方法可以避免一些不必要的计算

In [60]:
# apply： 一般性的“拆分－应用－合并”
# 最通用的 GroupBy 方法是 apply，apply 会将待处理的对象拆分成多个片段， 
#                 然后对各片段调用传入的函数， 最后尝试将各片段组合到一起
# as the picture shows

![self](./../image/groupby.png)

In [62]:
# 回到之前那个小费数据集， 假设想要根据分组选出最高的5个 tip_pct 值。 
# 首先， 编写一个选取指定列具有最大值的行的函数：
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [63]:
# 现在，对 smoker 分组并应用该函数调用 apply
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [64]:
# 这里发生了什么？ 
# top 函数在 DataFrame 的各个片段上调用， 然后结果由 pandas.concat 组装到一起， 并以分组名称进行了标记
#  于是， 最终结果就有了一个层次化索引， 其内层索引值来自原 DataFrame。

# 如果传给 apply 的函数能够接受其他参数或关键字， 则可以将这些内容放在函数名后面一并传
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [65]:
# 笔记： 除这些基本用法之外， 能否充分发挥 apply 的威力很大程度上取决于你的创造力
#      传入的那个函数能做什么全由你说了算， 它只需返回一个 pandas 对象 或 标量值 即可

# 示例主要用于讲解如何利用 groupby 解决各种各样的问题

# 想起来了， 之前在 GroupBy 对象上调用过 describe：
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [66]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [67]:
# 在 GroupBy 中，当调用诸如 describe 之类的方法时， 
# 实际上只是应用了下面两条代码的快捷方式而已：
# f = lambda x: x.describe()
# grouped.apply(f)

In [68]:
# 禁止分组键

# 从上面的例子中可以看出， 分组键会跟原始对象的索引共同构成结果对象中的层次化索引
#     将 group_keys=False 传入 groupby 即可禁止该效果：
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [71]:
# 分位数和桶分析

#  pandas 有一些能根据指定面元或样本分位数将数据拆分成多块的工具（比如 cut和qcut）
#    将这些函数跟 groupby 结合起来， 就能非常轻松地实现对数据集的桶（bucket） 或分位数（quantile）分析了

# 下面这个简单的随机数据集，利用 cut 将其装入长度相等的桶中：
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0     (-0.0678, 1.593]
1    (-1.729, -0.0678]
2     (-0.0678, 1.593]
3     (-0.0678, 1.593]
4    (-1.729, -0.0678]
5     (-0.0678, 1.593]
6     (-0.0678, 1.593]
7     (-0.0678, 1.593]
8    (-1.729, -0.0678]
9     (-0.0678, 1.593]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.396, -1.729] < (-1.729, -0.0678] < (-0.0678, 1.593] < (1.593, 3.254]]

In [72]:
# 由 cut 返回的 Categorical 对象可直接传递到 groupby
#  因此，可以像下面这样对 data2 列做一些 统计计算：
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.396, -1.729]",-2.075586,1.682228,38.0,0.065943
"(-1.729, -0.0678]",-2.933403,2.546406,400.0,-0.078434
"(-0.0678, 1.593]",-3.315246,3.154464,504.0,-0.015595
"(1.593, 3.254]",-2.341893,2.881565,58.0,0.021167


In [73]:
# 这些都是长度相等的桶。 
# 要根据样本分位数得到大小相等的桶，使用 qcut 即可
#   传入 labels=False  即可只获取分位数的编号：

# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.804969,2.062202,100.0,0.030912
1,-2.298309,2.075956,100.0,-0.240239
2,-2.355418,2.546406,100.0,-0.090931
3,-2.264593,2.297343,100.0,-0.031752
4,-3.315246,2.709260,100.0,-0.003540
5,-2.196621,2.980462,100.0,0.035718
6,-2.407073,2.045195,100.0,-0.127407
7,-2.243753,3.154464,100.0,-0.032576
8,-2.534372,2.703518,100.0,0.114540
